In [ ]:
## ---------------- SEP5 testing gedi extraction code -----------------------

In [ ]:
##
# conda install -c conda-forge r-optmatch
##

In [44]:
install.packages("s3")
install.packages("RItools")    

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [45]:
library("terra")
library("dplyr")
library("sf")
#install.packages("s3")
library("s3")

library("sp")
library("foreach")
library("stringr")
library("aws.s3")
#conda install -c conda-forge r-optmatch #r-ggmap r-hrbrthemes r-Hmisc
library("optmatch")
#install.packages("doParallel")
library("doParallel")


In [46]:
s3 <- paws::s3()

In [47]:
#To test, we define the variables manually. For final version, run the commented out section below
#iso3 <-"Bpt"
iso3 <-"Bpt"
gediwk <- 24
mproc <- 1

In [48]:
#-------------------------------------------------------------------------------
#args = commandArgs(trailingOnly=TRUE)
#if (length(args)==0) {
#  stop("At least one argument must be supplied (input file).n", call.=FALSE)
#} else if (length(args)>=1) {
#  
#  iso3 <- args[1]  #country to process
#  gediwk <- args[2]   #the # of weeks GEDI data to use
#  mproc <- as.integer(args[3])  #the number of cores to use for matching 
#}
#-------------------------------------------------------------------------------

In [49]:
cat("Step 0: Loading global variables for", iso3,"with wk", gediwk, "data \n")

#f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"

matching_tifs <- c("wwf_biomes","wwf_ecoreg","lc2000","d2roads", "dcities","dem",
                   "pop_cnt_2000","pop_den_2000","slope", "tt2cities_2000", "wc_prec_1990-1999",
                   "wc_tmax_1990-1999","wc_tavg_1990-1999","wc_tmin_1990-1999" )

ecoreg_key <- read.csv(s3_get(paste(f.path,"wwf_ecoregions_key.csv",sep="")))
#unlink(s3_get(paste(f.path,"wwf_ecoregions_key.csv",sep="")))

allPAs <- readRDS(s3_get(paste(f.path,"WDPA_shapefiles/WDPA_polygons/",iso3,"_PA_poly.rds",sep="")))

MCD12Q1 <- rast(s3_get(paste(f.path,"GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif",sep="")))
crs(MCD12Q1)  <- "epsg:6933"

world_region <- rast(s3_get(paste(f.path,"GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif",sep="")))
crs(world_region)  <- "epsg:6933"

s3_get_files(c(paste(f.path,"WDPA_countries/shp/",iso3,".shp",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".shx",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".prj",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".dbf",sep="")),confirm = FALSE)
adm <- st_read(s3_get(paste(f.path,"WDPA_countries/shp/",iso3,".shp",sep="")))

#s3_path <- paste("/vsis3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_countries/shp/",iso3,".shp",sep="") #Redo this for the gpkg
#adm <- st_read(s3_path)
adm_prj <- project(vect(adm), "epsg:6933")

load(s3_get(paste(f.path,"rf_noclimate.RData",sep="")))
#source(s3_get(paste(f.path,"matching_func.R",sep="")))
source(s3_get(paste(f.path,"matching_func_2024.R",sep="")))

flag <- "run all"

Step 0: Loading global variables for Bpt with wk 24 data 


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/wwf_ecoregions_key.csv already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/wwf_ecoregions_key.csv

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_shapefiles/WDPA_polygons/Bpt_PA_poly.rds already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_shapefiles/WDPA_polygons/Bpt_PA_poly.rds

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_CONTINE

Reading layer `Bpt' from data source 
  `/projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_countries/shp/Bpt.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 7 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -59.18886 ymin: -22.11834 xmax: -54.92231 ymax: -15.52386
Geodetic CRS:  WGS 84


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/rf_noclimate.RData already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/rf_noclimate.RData

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/matching_func_2024.R already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/matching_func_2024.R



In [50]:
#---------------STEP5. GEDI PROCESSING ---------------- 
#using GEDI shots to extract the treatment/control status, also extract the MODIS PFT for AGB prediction

# if (file.exists(paste(f.path,"WDPA_GEDI_extract/",iso3,"_wk",gediwk,"/",iso3,"_gedi_extracted_matching_wk",gediwk,".RDS", sep=""))){
cat(paste("Step 5: Performing WK ", gediwk, "GEDI extraction for", iso3,"\n"))
#matched_all <-read.csv(paste(f.path,"WDPA_extract4_residual_PAs/", iso3, "_wk_", gediwk, "_null_matches_rerun.csv",sep="")) 
matched_all <- read.csv(s3_get(paste(f.path,"WDPA_matching_results/",iso3,"_wk24_filelist.csv",sep="")))[,2]
matched_all
     

Step 5: Performing WK  24 GEDI extraction for Bpt 


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24_filelist.csv already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_matching_results/Bpt_wk24_filelist.csv



[1] "Bpt_pa_220297_matching_results_wk24.RDS"   
 [2] "Bpt_pa_2220_matching_results_wk24.RDS"     
 [3] "Bpt_pa_2581_matching_results_wk24.RDS"     
 [4] "Bpt_pa_33968_matching_results_wk24.RDS"    
 [5] "Bpt_pa_33985_matching_results_wk24.RDS"    
 [6] "Bpt_pa_352016_matching_results_wk24.RDS"   
 [7] "Bpt_pa_352021_matching_results_wk24.RDS"   
 [8] "Bpt_pa_352270_matching_results_wk24.RDS"   
 [9] "Bpt_pa_352271_matching_results_wk24.RDS"   
[10] "Bpt_pa_352430_matching_results_wk24.RDS"   
[11] "Bpt_pa_352461_matching_results_wk24.RDS"   
[12] "Bpt_pa_352463_matching_results_wk24.RDS"   
[13] "Bpt_pa_352466_matching_results_wk24.RDS"   
[14] "Bpt_pa_478452_matching_results_wk24.RDS"   
[15] "Bpt_pa_555542500_matching_results_wk24.RDS"
[16] "Bpt_pa_555542528_matching_results_wk24.RDS"
[17] "Bpt_pa_555542552_matching_results_wk24.RDS"
[18] "Bpt_pa_555542777_matching_results_wk24.RDS"
[19] "Bpt_pa_555576153_matching_results_wk24.RDS"
[20] "Bpt_pa_555576191_matching_results_wk24.RDS"
[21] "Bpt_pa_555576276_matching_results_wk24.RDS"
[22] "Bpt_pa_555576295_matching_results_wk24.RDS"
[23] "Bpt_pa_555576310_matching_results_wk24.RDS"
[24] "Bpt_pa_555576396_matching_results_wk24.RDS"
[25] "Bpt_pa_555576444_matching_results_wk24.RDS"
[26] "Bpt_pa_555599959_matching_results_wk24.RDS"
[27] "Bpt_pa_555600164_matching_results_wk24.RDS"
[28] "Bpt_pa_555682279_matching_results_wk24.RDS"
[29] "Bpt_pa_67816_matching_results_wk24.RDS"

In [51]:
f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"

matched_PAs <- foreach(this_rds=matched_all, .combine = c, .packages=c('sp','magrittr', 'dplyr','tidyr','terra')) %do% {  #non-NA matched results
  matched_PAs=c()
  print(this_rds)
  if(nchar(iso3)>3){
    id_pa <- basename(this_rds)%>%readr::parse_number() %>% unique()  
  } else {
    id_pa <- basename(this_rds)%>%readr::parse_number() %>% unique()
  }
  matched <- readRDS(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",iso3,"_pa_",id_pa,"_matching_results_wk",gediwk,".RDS",sep=""))
  if(!is.null(matched)){
    if(nrow(matched)!=0){
      matched_PAs=c(matched_PAs,this_rds) 
    }
  }else {
    # print(this_rds)
    matched_PAs=matched_PAs
  }
  return(matched_PAs)
}
length(matched_PAs)

[1] "Bpt_pa_220297_matching_results_wk24.RDS"
[1] "Bpt_pa_2220_matching_results_wk24.RDS"
[1] "Bpt_pa_2581_matching_results_wk24.RDS"
[1] "Bpt_pa_33968_matching_results_wk24.RDS"
[1] "Bpt_pa_33985_matching_results_wk24.RDS"
[1] "Bpt_pa_352016_matching_results_wk24.RDS"
[1] "Bpt_pa_352021_matching_results_wk24.RDS"
[1] "Bpt_pa_352270_matching_results_wk24.RDS"
[1] "Bpt_pa_352271_matching_results_wk24.RDS"
[1] "Bpt_pa_352430_matching_results_wk24.RDS"
[1] "Bpt_pa_352461_matching_results_wk24.RDS"
[1] "Bpt_pa_352463_matching_results_wk24.RDS"
[1] "Bpt_pa_352466_matching_results_wk24.RDS"
[1] "Bpt_pa_478452_matching_results_wk24.RDS"
[1] "Bpt_pa_555542500_matching_results_wk24.RDS"
[1] "Bpt_pa_555542528_matching_results_wk24.RDS"
[1] "Bpt_pa_555542552_matching_results_wk24.RDS"
[1] "Bpt_pa_555542777_matching_results_wk24.RDS"
[1] "Bpt_pa_555576153_matching_results_wk24.RDS"
[1] "Bpt_pa_555576191_matching_results_wk24.RDS"
[1] "Bpt_pa_555576276_matching_results_wk24.RDS"
[1] "Bpt_pa_5555762

[1] 26

In [52]:
if(flag=="run all"){  #determine how many PAs to run the extraction process
  matched_PAs <- matched_PAs
  cat("Step 5: runing extraction on all", length(matched_PAs),"of non-NA matched results in", iso3,"\n")
} else if (flag=="run remaining"){
  pattern1 = c(paste("wk",gediwk,sep=""),"RDS")
  extracted_PAid <- list.files(paste(f.path3,"WDPA_GEDI_extract/",iso3,"_wk",gediwk,"/",sep=""), full.names = F, pattern=paste0(pattern1, collapse="|"))%>%
    readr::parse_number() %>% unique()
  matched_PA_id <- matched_PAs %>% readr::parse_number()
  runPA_id <- matched_PA_id[!(matched_PA_id %in% extracted_PAid)]
  if (length(runPA_id)>0){
    Pattern2 <-  paste(runPA_id, collapse="|")
    runPA <-  matched_PAs[grepl(Pattern2,matched_PAs)]
    # runPA_ind <- str_detect(matched_PAs, paste(runPA_id, collapse = "|"))
    matched_PAs <-runPA
  } else {
    matched_PAs <- NULL
    cat("Step 5 already done for", iso3, "\n")
  }
}

Step 5: runing extraction on all 26 of non-NA matched results in Bpt 


In [53]:
matched_PAs[1]

[1] "Bpt_pa_220297_matching_results_wk24.RDS"

In [57]:
matched2ras(matched)

Bpt converting the matched csv to a raster stack for extraction


ERROR: Error in extent(buffer(matched_pts, 10000)): could not find function "extent"


In [55]:
this_rds <- matched_PAs[1]
id_pa <- basename(this_rds) %>% readr::parse_number() %>% unique()
    
    matched <- readRDS(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",iso3,"_pa_",id_pa,"_matching_results_wk",gediwk,".RDS",sep=""))
  
    if (is.null(matched)==TRUE  | nrow(matched)==0) {
        cat("Matched result is null for PA", id_pa, "quitting...\n")
        next  # Skip to the next iteration
    } else if (!is.null(matched)==TRUE){
    mras  <- tryCatch(matched2ras(matched),
                      error=function(cond){
                        message(cond)
                        cat("Matched result is likely null for country", iso3,"pa", id_pa, "dimension of the match is", dim(matched),"\n")
                        # writeLines("Matched results is likely null for country", paste(f.path,"WDPA_log/",iso3,"_log_matching.txt", sep=""))
                        return(NULL)}) #convert the macthed df to a raster stack 
    print(table(mras$status[]))
    if(table(mras$status[])[2]==0 | table(mras$status[])[1]==0 | is.null(mras)){
      cat("Rasterized results unbalanced for PA", id_pa, "quitting...\n")
      next  # Skip to the next iteration
    } else {
        
      startTime <- Sys.time()
        
      iso_matched_gedi <- extract_gedi(matched=matched, mras = mras)#}  #run filtered csvs on mras for extarction
        
        if (is.null(iso_matched_gedi)) {
        cat("Matched result is null for PA", id_pa, "quitting...\n")
        next  # Skip to the next iteration
        }

      tElapsed <- Sys.time()-startTime
      cat(tElapsed, "for extracting all PAs in", iso3,"\n")
      cat("Done GEDI for no. ",grep(unique(matched$pa_id), matched_PAs),"pa out of", length(matched_PAs),"\n")

      iso_matched_gedi <-  iso_matched_gedi %>%
            dplyr::select("pa_id","status",
                      "wwfbiom","wwfecoreg","shot_number","lon_lowestmode", 
                      "lat_lowestmode", "lon_lowestmode", 
                      "lat_lowestmode","rh25", "rh50", "rh75","rh90", "rh98")  #write to individual country folder
        
      if (length(unique(iso_matched_gedi$wwfbiom)) >1){
#       pabiome <- iso_matched_gedi$wwfbiom %>% unique() %>% gsub("\b(\pL)\pL{2,}|.","\U\1",.,perl = TRUE)%>% str_c( collapse = "+")
        pabiome <- iso_matched_gedi$wwfbiom %>% unique() %>% str_replace_all("\\b(\\p{L})\\p{L}{2,}|.", "\\U\\1", perl = TRUE) %>% str_c(collapse = "+")
      } else if (length(unique(iso_matched_gedi$wwfbiom))==1){
        pabiome <- iso_matched_gedi$wwfbiom %>% unique() %>% str_replace_all("\\b(\\p{L})\\p{L}{2,}|.", "\\U\\1", perl = TRUE)
      } else {
        pabiome <- iso_matched_gedi$wwfbiom %>% unique() }
        continent <- unique(iso_matched_gedi$region) %>% getmode()
      print(paste('output df',dim(iso_matched_gedi)))

      dir.create(file.path(paste(f.path,"WDPA_GEDI_extract/",sep="")),recursive=TRUE)
      saveRDS(iso_matched_gedi, file=paste(f.path,"WDPA_GEDI_extract/",iso3,"_pa_", id_pa,"_gedi_wk_",gediwk,"_conti_","biome_",pabiome,".RDS", sep=""))
      write.csv(iso_matched_gedi, file=paste(f.path,"WDPA_GEDI_extract/",iso3,"_pa_", id_pa,"_iso_matched_gedi_sub_wk_",gediwk,".csv", sep=""))
      cat(id_pa,"in",iso3,"result is written to dir\n")
    }
    }
    
    return(NULL)

Bpt converting the matched csv to a raster stack for extraction


ERROR: Error in extent(buffer(matched_pts, 10000)): could not find function "extent"


In [ ]:
dim(gedi_l24)

In [ ]:
iso_matched_gedi_df <- data.frame()

gedi_l24_sp <- gedi_l24 %>% 
     SpatialPointsDataFrame(coords=.[,c("lon_lowestmode","lat_lowestmode")],
                                     proj4string=CRS("+init=epsg:4326"), data=.) %>%spTransform(., CRS("+init=epsg:6933"))
   

In [ ]:
matched_gedi <- terra::extract(mras,vect(gedi_l24_sp), df=TRUE)


In [ ]:
plot(gedi_l24_sp)


In [ ]:
plot(mras$status)
plot(gedi_l24_sp,add=T)

In [ ]:
allOut <- read.csv("~/GEDI_PA/vl_GEDI-PA_2024/GNB_pa.csv")

In [ ]:
data_sf <- st_as_sf(allOut, coords = c("lon", "lat"), crs = 4326)


In [ ]:
results <- s3$list_objects_v2(Bucket = "maap-ops-workspace", 
                            Prefix=paste("shared/abarenblitt/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles/",sep=""))
    all_gedil2_f <- sapply(results$Contents, function(x) {x$Key})
    pattern=paste(".gpkg",sep="")
    all_gedil2_f <- grep(pattern, all_gedil2_f, value=TRUE)
    all_gedil2_f <- basename(all_gedil2_f)[4:5] #Currently specifying working files
    
    results4 <- s3$list_objects_v2(Bucket = "maap-ops-workspace", 
                                Prefix=paste("shared/abarenblitt/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/",sep=""))
    all_gedil4_f <- sapply(results4$Contents, function(x) {x$Key})
    pattern4=paste(".gpkg",sep="")
    all_gedil4_f <- grep(pattern4, all_gedil4_f, value=TRUE)
    all_gedil4_f <- basename(all_gedil4_f)[4:5] #Currently specifying working files


In [ ]:
all_gedil2_f

In [ ]:
gedil4_f <- as.data.frame(st_read(paste(gedipath,"WDPA_gedi_L4A_tiles/",all_gedil4_f[this_csvid],sep="")))

In [ ]:
gedil2_f <- as.data.frame(st_read(paste(gedipath,"WDPA_gedi_L2A_tiles/",all_gedil2_f[this_csvid],sep="")))

In [ ]:
results <- s3$list_objects_v2(Bucket = "maap-ops-workspace", 
                            Prefix=paste("shared/abarenblitt/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles/",sep=""))
    all_gedil2_f <- sapply(results$Contents, function(x) {x$Key})
    pattern=paste(".gpkg",sep="")
    all_gedil2_f <- grep(pattern, all_gedil2_f, value=TRUE)
    all_gedil2_f <- basename(all_gedil2_f)[4:5]

all_gedil2_f

In [ ]:
extract_gedi <- function(matched, mras){
    results <- s3$list_objects_v2(Bucket = "maap-ops-workspace", 
                            Prefix=paste("shared/abarenblitt/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles/",sep=""))
    all_gedil2_f <- sapply(results$Contents, function(x) {x$Key})
    pattern=paste(".gpkg",sep="")
    all_gedil2_f <- grep(pattern, all_gedil2_f, value=TRUE)
    all_gedil2_f <- basename(all_gedil2_f)[4:5] #Currently specifying working files
    
    results4 <- s3$list_objects_v2(Bucket = "maap-ops-workspace", 
                                Prefix=paste("shared/abarenblitt/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/",sep=""))
    all_gedil4_f <- sapply(results4$Contents, function(x) {x$Key})
    pattern4=paste(".gpkg",sep="")
    all_gedil4_f <- grep(pattern4, all_gedil4_f, value=TRUE)
    all_gedil4_f <- basename(all_gedil4_f)[4:5] #Currently specifying working files
  
    # Initialize an empty list to store results
    results_list <- list()
    iso_matched_gedi_df <- NULL # Initialize before loop

            # Iterate over the sequence of indices for your files
    for (this_csvid in seq_along(all_gedil2_f)) {
                cat("Reading in no. ", this_csvid, "csv of ", length(all_gedil2_f), "csvs for iso3", iso3, "\n")
                
                # Read GEDI L4A data
                gedil4_f_path <- paste(gedipath, "WDPA_gedi_L4A_tiles/", all_gedil4_f[this_csvid], sep = "")
                gedil4_f <- as.data.frame(st_read(gedil4_f_path))
                
                # Read GEDI L2A data
                gedil2_f_path <- paste(gedipath, "WDPA_gedi_L2A_tiles/", all_gedil2_f[this_csvid], sep = "")
                gedil2_f <- as.data.frame(st_read(gedil2_f_path))
            
                # Check if GEDI L4A data is empty
                if (nrow(gedil4_f) < 1) {
                    cat("Error: No data for GEDI L4A\n")
                    gedi_l24 <- gedil2_f
                    gedi_l24
agbd_se <- NA
                    gedi_l24
agbd_t_se <- NA
                } else {
                    # Select relevant columns from GEDI L4A
                    gedi_l4_sub <- gedil4_f %>%
                        dplyr::select(shot_number, agbd, agbd_se, agbd_t, agbd_t_se)
                    
                    # Join with GEDI L2A data
                    gedi_l24 <- inner_join(gedil2_f, gedi_l4_sub, by = "shot_number")
                }
            
                print(dim(gedi_l24))
            
                # Initialize empty spatial object for the current iteration
                gedi_l24_sp <- NULL
            
                # Convert to spatial points data frame if there is data
                if (nrow(gedi_l24) > 0) {
                    gedi_l24_sp <- SpatialPointsDataFrame(
                        coords = gedi_l24[, c("lon_lowestmode", "lat_lowestmode")],
                        data = gedi_l24,
                        proj4string = CRS("+init=epsg:4326")
                    ) %>% spTransform(CRS("+init=epsg:6933"))
                matched_gedi <- terra::extract(mras,vect(gedi_l24_sp), df=TRUE)
                matched_gedi_metrics <- cbind(matched_gedi,gedi_l24_sp@data)
                matched_gedi_metrics_filtered <- matched_gedi_metrics %>% dplyr::filter(!is.na(status)) %>% 
                convertFactor(matched0 = matched,exgedi = .) 

            iso_matched_gedi_df <- rbind(matched_gedi_metrics_filtered,iso_matched_gedi_df)
            print(dim(iso_matched_gedi_df))
         }
        
        # Store results in a list
        results_list[[this_csvid]] <- iso_matched_gedi_df
    }
    
    # Combine all results
    if (!is.null(iso_matched_gedi_df)) {
        iso_matched_gedi_df <- do.call(rbind, results_list)
    }
    
    cat("Done GEDI processing\n")
    return(iso_matched_gedi_df)
}
     

In [ ]:
ex_out <- for (this_csvid in seq_along(length(all_gedil2_f))) {
        ##add the GEDI l4a model prediction for AGB here :
        cat("Readng in no. ", this_csvid,"csv of ", length(all_gedil2_f),"csvs for iso3",iso3,"\n")

    # for (this_gedi4 in all_gedil4_f[this_csvid]){
            gedil4_f <- as.data.frame(st_read(paste(gedipath,"WDPA_gedi_L4A_tiles/",all_gedil4_f[this_csvid],sep="")))
    # }
    # for (this_gedi2 in all_gedil2_f[this_csvid]){
            gedil2_f <- as.data.frame(st_read(paste(gedipath,"WDPA_gedi_L2A_tiles/",all_gedil2_f[this_csvid],sep="")))
    # }
            if (nrow(gedil4_f) < 1){   #is.na(gedi_l4) || 
              cat("error")
              gedi_l24 <- gedil2_f
              gedi_l24
agbd_se <- NA
              gedi_l24
agbd_t_se <- NA
            } else {
              gedi_l4_sub <- gedil4_f %>%
                dplyr::select(shot_number, agbd, agbd_se, agbd_t, agbd_t_se)
              gedi_l24 <- inner_join(gedil2_f, gedi_l4_sub, by="shot_number")

            }

        print(dim(gedi_l24))
        iso_matched_gedi_df <- data.frame()
        if(nrow(gedi_l24)>0){
            gedi_l24_sp <- gedi_l24 %>% 
                SpatialPointsDataFrame(coords=.[,c("lon_lowestmode","lat_lowestmode")],
                                     proj4string=CRS("+init=epsg:4326"), data=.) %>%spTransform(., CRS("+init=epsg:6933"))
        }
        return(gedi_l24_sp)
}
     

In [ ]:
results_list <- list()

# Iterate over the sequence of indices for your files
for (this_csvid in seq_along(all_gedil2_f)) {
    cat("Reading in no. ", this_csvid, "csv of ", length(all_gedil2_f), "csvs for iso3", iso3, "\n")
    
    # Read GEDI L4A data
    gedil4_f_path <- paste(gedipath, "WDPA_gedi_L4A_tiles/", all_gedil4_f[this_csvid], sep = "")
    gedil4_f <- as.data.frame(st_read(gedil4_f_path))
    
    # Read GEDI L2A data
    gedil2_f_path <- paste(gedipath, "WDPA_gedi_L2A_tiles/", all_gedil2_f[this_csvid], sep = "")
    gedil2_f <- as.data.frame(st_read(gedil2_f_path))

    # Check if GEDI L4A data is empty
    if (nrow(gedil4_f) < 1) {
        cat("Error: No data for GEDI L4A\n")
        gedi_l24 <- gedil4_f
        gedi_l24
agbd_se <- NA
        gedi_l24
agbd_t_se <- NA
    } else {
        # Select relevant columns from GEDI L4A
        gedi_l4_sub <- gedil4_f %>%
            dplyr::select(shot_number, agbd, agbd_se, agbd_t, agbd_t_se)
        
        # Join with GEDI L2A data
        gedi_l24 <- inner_join(gedil2_f, gedi_l4_sub, by = "shot_number")
    }

    print(dim(gedi_l24))

    # Initialize empty spatial object for the current iteration
    gedi_l24_sp <- NULL

    # Convert to spatial points data frame if there is data
    if (nrow(gedi_l24) > 0) {
        gedi_l24_sp <- SpatialPointsDataFrame(
            coords = gedi_l24[, c("lon_lowestmode", "lat_lowestmode")],
            data = gedi_l24,
            proj4string = CRS("+init=epsg:4326")
        ) %>% spTransform(CRS("+init=epsg:6933"))
    }
    
    # Store result in the list
    results_list[[this_csvid]] <- gedi_l24_sp
}

In [ ]:
single_result <- results_list[[2]]


In [ ]:
single_result_sf <- st_as_sf(single_result)


In [ ]:
matched<-readRDS("~/output/WDPA_matching_results/GNB_wk24/GNB_pa_351088_matching_results_wk24.RDS")


In [ ]:
ggplot(data_sf) +
  geom_sf(aes(color = status), size = 1) + 
  geom_sf(data = single_result_sf, color = "black", linetype = "dashed") +
  theme_minimal() +
  scale_color_manual(values = c("TRUE" = "purple", "FALSE" = "blue"))+
  labs(title = "Spatial Points from CSV",
       x = "Longitude",
       y = "Latitude",
      color = "Status")
     


In [ ]:
all_gedil2_f


In [ ]:
gediGrid<-st_read("~/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles/tile_num_29599_L2A.gpkg")
